In [1]:
import time
import numpy as np
import pyroomacoustics as pra
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [2]:
# fname_1 = 'original_sound/borya_Karim.wav'
# fname_2 = 'original_sound/motivation_Aynur.wav'
fname_1 = 'original_sound/borya_Karim.wav'
fname_2 = 'original_sound/motivation_Aynur.wav'

In [3]:
fs, audio_in_1 = wavfile.read(fname_1)
fs, audio_in_2 = wavfile.read(fname_2)

In [4]:
print(audio_in_1.shape)
print(audio_in_2.shape)

(1793547,)
(2268945,)


In [5]:
fs

44100

In [6]:
if audio_in_1.shape[0] > audio_in_2.shape[0]:
    audio_in_2 = np.concatenate((np.zeros(audio_in_1.shape[0] - audio_in_2.shape[0]), audio_in_2))
else:
    audio_in_1 = np.concatenate((np.zeros(audio_in_2.shape[0] - audio_in_1.shape[0]), audio_in_1))

In [7]:
print(audio_in_1.shape)
print(audio_in_2.shape)

(2268945,)
(2268945,)


### Normalizing sources

In [8]:

audio_in_1 = audio_in_1 - np.mean(audio_in_1, axis = 0)
audio_in_1 = audio_in_1 / np.std(audio_in_1, axis=0)

audio_in_2 = audio_in_2 - np.mean(audio_in_2, axis = 0)
audio_in_2 = audio_in_2 / np.std(audio_in_2, axis=0)

### Simulation

In [9]:
absorption = 1.0
room_dim = [5, 4, 3]

In [10]:
# Create the shoebox
shoebox = pra.ShoeBox(
    room_dim,
    absorption=absorption,
    fs=fs,
    max_order=15,
    )

# source and mic locations
shoebox.add_source([2, 1, 1], signal=audio_in_1)
shoebox.add_source([3, 1, 1], signal=audio_in_2)


# run ism
R = np.array([[2, 3], 
              [2, 2], 
              [1, 1]])

In [11]:
shoebox.add_microphone_array(pra.MicrophoneArray(R, shoebox.fs))
shoebox.simulate()

In [12]:
audio_reverb = shoebox.mic_array.to_wav('mixed/' 
                                        + fname_1.split('/')[1] + '_' 
                                        + fname_2.split('/')[1] + '_absorb='+str(absorption) 
                                        + '.wav', norm=True, bitdepth=np.int16)